# Adding MTP Dependencies to a Slice

## Input Required Information

If you do not already have a slice created, please either create one using your own method or via the GraphML Slice Builder notebook provided. Also remember to configure your local Jupyter enviornment if you have not already (can you reach out to the nodes?).

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of the slice you wish to work on. |

In [ ]:
SLICE_NAME = "routers"
CLIENT_PREFIX = "C"

## Access the Slice

FabOrchestrator is a custom Python class that utilizes the existing FabLib API. Its purpose is to make repetitive code easier to manage and build around features that we previously used in our custom GENI orchestrator.

The orchestrator class is initalized here, which also means the slice and its nodes are now accessable as well..

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import json
 
# Opening JSON file
f = open(f'{SLICE_NAME}_slice_log.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

## Upload Configuration Script

Anything that would make sense to be configured that does not include package installation should be placed in the init_mtp.sh script. Currently, this modifies the tmux configuration after it is installed. 

In [ ]:
configScript = "/home/fabric/work/custom/FABRIC-Automation/remote_scripts/mtp_scripts/init_mtp.sh"
manager.uploadFileParallel(configScript, prefixList="L,S")

## Install Dependencies and Make Configuration Changes on all Nodes

Along with installing necessary pacakges, the initialization script uploaded in the prior cell is also run.

| Package | Use |
| --- | --- |
| tmux    | Terminal multiplexer similar to GNU Screen to allow for ssh sessions to disconnect and let the process continue to run. |
| Wireshark | Access to tshark, the command-line based version of the packet sniffer. |
| Development Tools | Includes all necessary applications, such as GNU GCC C compiler, make, debuggers, man pages. etc., which are needed to compile, build, and troubleshoot MTP code |

In [ ]:
# The commands to run
packagesToInstall = "sudo dnf install -q -y tmux wireshark"
devToolsInstall = 'sudo dnf groupinstall -q -y "Development Tools"'
config = 'bash init_mtp.sh {name}'

# Execute the commands
manager.executeCommandsParallel(packagesToInstall)
manager.executeCommandsParallel(devToolsInstall)
manager.executeCommandsParallel(config, prefixList="L,S", addNodeName=True)

## Add Default Routes to the Compute Nodes

The core data center network (DCN) nodes will forward based on the MTP data-plane header. Compute nodes are MTP-unaware and still require standard IP-based forwarding. This includes the ability to route outside of its LAN. The leaf interface which attaches the compute node to the rest of the DCN is set as the default gateway. These interfaces on each leaf are set to 254 for the last octet.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import re

SLICE_SUPERNET = "192.168.0.0/16"

for computeNode in manager.selectedNodes(CLIENT_PREFIX):
    # Get the compute node's interface
    intfName = f"{computeNode.get_name()}-eth1-p1"
    intf = computeNode.get_interface(intfName)
    
    # Get the interface IPv4 address and its third octet
    ipAddress = intf.get_ip_addr()
    IPGroup = re.search(r"192\.168\.([0-9]{1,3})\.[0-9]{1,3}", ipAddress)
    thirdOctet = IPGroup.group(1)
    nextHop = f"192.168.{thirdOctet}.254"
    
    # Add the route to the node
    computeNode.ip_route_add(subnet=IPv4Network(SLICE_SUPERNET), gateway=IPv4Address(nextHop))
    
    print(f"Adding route {SLICE_SUPERNET} to {computeNode.get_name()} with next-hop {nextHop}")

## Turn off Traditional IP-based Forwarding on MTP Nodes

This is not done on compute/client nodes, as they are MTP-unaware.

In [ ]:
cmdOff = "sudo sysctl -w net.ipv4.ip_forward=0"
cmdOn = "sudo sysctl -w net.ipv4.ip_forward=1"
manager.executeCommandsParallel(cmdOff, prefixList="L,S")
manager.executeCommandsParallel(cmdOn, prefixList="C")